# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 

for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
                
        for line in csvreader:
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra Queries

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId



<img src="images/image_event_datafile_new.jpg">

In [5]:
df = pd.read_csv('event_datafile_new.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
artist           6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
sessionId        6820 non-null int64
song             6820 non-null object
userId           6820 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB


#### Creating a Cluster

In [6]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

session = cluster.connect()

#### Create Keyspace

In [7]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

                    

In [9]:
query = "DROP TABLE IF EXISTS music_app_history_session"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [10]:
query = "CREATE TABLE IF NOT EXISTS music_app_history_session"
query = query + "(session_id int,item_in_session int,artist text,song text,length decimal, \
PRIMARY KEY(session_id, item_in_session))"  
try:
    session.execute(query)
except Exception as e:
    print(e)

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO music_app_history_session(session_id,item_in_session,artist,song,length)"
        query = query + "VALUES(%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]),int(line[3]),line[0], line[9],float(line[5])))

In [12]:
query = "SELECT artist,song,length from music_app_history_session \
WHERE session_id = 338 and item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


> In the first queries ask for the information where there is a particular sessionid and item in session, therefore all the songs have a particular sessionid as a partition key, and every time user play a new song there is a item_in_session as clustering column.

## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


                    

In [13]:
query = "DROP TABLE IF EXISTS music_app_history_artist"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [14]:
query = "CREATE TABLE IF NOT EXISTS music_app_history_artist"
query = query + "(userId int,session_id int,item_in_session int,artist text,song text,\
first_name text,last_name text, PRIMARY KEY((userId,session_id),item_in_session))"  
try:
    session.execute(query)
except Exception as e:
    print(e)

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO music_app_history_artist(userId,session_id,item_in_session,artist,\
        song,first_name,last_name)"
        query = query + "VALUES(%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query,(int(line[10]),int(line[8]),int(line[3]),line[0], line[9],line[1],line[4]))

In [16]:
query = "SELECT artist,song,first_name,last_name from music_app_history_artist \
WHERE userId = 10 AND session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.first_name,row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


> First, we need the queries to order by item_in_session, there is item_in_session is a clustering column,and we have to use session_id and userid as unique key which is the partition key to query because like I said in the previous query, sessionid is unique to every session, and userid is also unique to every user. 

## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


                    

In [17]:
query = "DROP TABLE IF EXISTS music_app_history_user"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [18]:
query = "CREATE TABLE IF NOT EXISTS music_app_history_user"
query = query + "(song text,userId int,artist text,first_name text,last_name text, \
PRIMARY KEY(song,userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [19]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO music_app_history_user(song,userId,artist,first_name,last_name)"
        query = query + "VALUES(%s,%s,%s,%s,%s)"
        session.execute(query,(line[9],int(line[10]),line[0],line[1],line[4]))

In [20]:
query = "SELECT first_name,last_name from music_app_history_user \
WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.first_name,row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


> Here in this query, since we want all the user who listens to a particular song, song should be the partition key, and we also need to have userid as a culstering column to make sure we return all the user and does not result in "ALLOW FILTERING" warning. 

### Drop the tables before closing out the sessions

In [21]:
query = "DROP TABLE IF EXISTS music_app_history_session"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [22]:
query = "DROP TABLE IF EXISTS music_app_history_artist"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [23]:
query = "DROP TABLE IF EXISTS music_app_history_user"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [24]:
session.shutdown()
cluster.shutdown()